In [1]:
import pandas as pd

In [3]:
df=pd.read_pickle('industry_labeled.pickle')

In [4]:
df

,날짜,제목,본문,링크,산업
9,2025-10-01 00:13:00,"캐너코드,""테슬라 3분기 인도 48만대 넘어""…목표주가도 상향",이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. ...,https://n.news.naver.com/mnews/article/015/000...,자동차
16,2025-10-01 00:14:00,트럼프 큰소리에도 美자동차 업체는 공장 구조조정중,이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. ...,https://n.news.naver.com/mnews/article/015/000...,자동차
19,2025-10-01 00:16:00,[이슈프리즘] 韓 제조업 미래는 누가 고민하나,“중국이 제조 혁신에 나섰다고 화웨이 같은 회사가 자동차 등 다른 분야에서 곧 나올...,https://n.news.naver.com/mnews/article/015/000...,자동차
47,2025-10-01 04:06:00,"""수소도시 만들어 수소산업 육성하자""",국회수소경제포럼 대표의원인 정태호 더불어민주당 의원이 수소산업 경쟁력 확보를 위한 ...,https://n.news.naver.com/mnews/article/008/000...,자동차
48,2025-10-01 04:10:00,"국토부, 서울 중화5구역 점검… '공공 정비사업' 속도낸다",공공성 강화 중심의 '9·7 공급대책'을 내놓은 정부가 공공정비사업 사업성을 확대하...,https://n.news.naver.com/mnews/article/008/000...,건설
...,...,...,...,...,...
206635,2025-12-21 18:29:00,3200명 정신없는 건설현장… AI·통합관제로 안전 지킨다[현장르포],지난 19일 찾은 서울 서초구 디에이치 클래스트(반포주공1단지 1·2·4주구 재건축...,https://n.news.naver.com/mnews/article/014/000...,건설
206751,2025-12-21 19:08:00,[AI헬스케어] 정부 주도 AI의료 기술 개발...닥터앤서3.0 참여 기업은,[이데일리 김승권 기자] 정부 주도 의료 인공지능(AI) 사업 ‘닥터앤서’가 3.0...,https://n.news.naver.com/mnews/article/018/000...,헬스케어
206814,2025-12-21 20:52:00,살 뺄 때만 먹는 줄 알았는데…알고보니 '완전식품'인 고구마의 놀라운 효능 [건강UP],[서울경제] 몸·마음·생활… 무분별한 정보는 많고 건강해야 할 곳도 많습니다. ...,https://n.news.naver.com/mnews/article/011/000...,헬스케어
206831,2025-12-21 21:24:00,[미리보는 이데일리 신문]차세대 비만약 핵심기술 K바이오가 장악,[이데일리 박지애 기자] 다음은 22일자 이데일리 신문 주요 기사다. △1면 -...,https://n.news.naver.com/mnews/article/018/000...,헬스케어


In [5]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# 1. 실제 존재하는 한국어 감성 분석 모델로 교체
# jaehyeong 모델
model_name = "jaehyeong/koelectra-base-v3-generalized-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to("cuda")

def analyze_sentiment_batch(texts, batch_size=64):
    model.eval()
    all_scores = []
    
    # tqdm으로 진행률 표시
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size].astype(str).tolist() # 문자열 변환
        
        # 토큰화 및 GPU 전송
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")
        
        with torch.no_grad():
            outputs = model(**inputs)
            # 확률값 계산 (Softmax)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            
            # 모델 출력 구조: [0: 부정 확률, 1: 긍정 확률]
            # 예: 긍정 0.9, 부정 0.1 이면 점수는 0.8
            batch_scores = (probs[:, 1] - probs[:, 0]).cpu().numpy()
            all_scores.extend(batch_scores)
            
    return all_scores

# 2. 데이터프레임 적용
# '본문' 컬럼 사용
df['sentiment_score'] = analyze_sentiment_batch(df['본문'])

# 결과 확인
print(df[['제목', 'sentiment_score']].head())

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [05:45<00:00,  1.07it/s]

                                    제목  sentiment_score
9   캐너코드,"테슬라 3분기 인도 48만대 넘어"…목표주가도 상향         0.855592
16         트럼프 큰소리에도 美자동차 업체는 공장 구조조정중         0.449221
19           [이슈프리즘] 韓 제조업 미래는 누가 고민하나         0.743420
47                "수소도시 만들어 수소산업 육성하자"         0.854099
48    국토부, 서울 중화5구역 점검… '공공 정비사업' 속도낸다         0.847423


In [14]:
df

,날짜,제목,본문,링크,산업,sentiment_score
9,2025-10-01,"캐너코드,""테슬라 3분기 인도 48만대 넘어""…목표주가도 상향",이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. ...,https://n.news.naver.com/mnews/article/015/000...,자동차,0.855592
16,2025-10-01,트럼프 큰소리에도 美자동차 업체는 공장 구조조정중,이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. ...,https://n.news.naver.com/mnews/article/015/000...,자동차,0.449221
19,2025-10-01,[이슈프리즘] 韓 제조업 미래는 누가 고민하나,“중국이 제조 혁신에 나섰다고 화웨이 같은 회사가 자동차 등 다른 분야에서 곧 나올...,https://n.news.naver.com/mnews/article/015/000...,자동차,0.743420
47,2025-10-01,"""수소도시 만들어 수소산업 육성하자""",국회수소경제포럼 대표의원인 정태호 더불어민주당 의원이 수소산업 경쟁력 확보를 위한 ...,https://n.news.naver.com/mnews/article/008/000...,자동차,0.854099
48,2025-10-01,"국토부, 서울 중화5구역 점검… '공공 정비사업' 속도낸다",공공성 강화 중심의 '9·7 공급대책'을 내놓은 정부가 공공정비사업 사업성을 확대하...,https://n.news.naver.com/mnews/article/008/000...,건설,0.847423
...,...,...,...,...,...,...
206635,2025-12-21,3200명 정신없는 건설현장… AI·통합관제로 안전 지킨다[현장르포],지난 19일 찾은 서울 서초구 디에이치 클래스트(반포주공1단지 1·2·4주구 재건축...,https://n.news.naver.com/mnews/article/014/000...,건설,0.963720
206751,2025-12-21,[AI헬스케어] 정부 주도 AI의료 기술 개발...닥터앤서3.0 참여 기업은,[이데일리 김승권 기자] 정부 주도 의료 인공지능(AI) 사업 ‘닥터앤서’가 3.0...,https://n.news.naver.com/mnews/article/018/000...,헬스케어,0.882646
206814,2025-12-21,살 뺄 때만 먹는 줄 알았는데…알고보니 '완전식품'인 고구마의 놀라운 효능 [건강UP],[서울경제] 몸·마음·생활… 무분별한 정보는 많고 건강해야 할 곳도 많습니다. ...,https://n.news.naver.com/mnews/article/011/000...,헬스케어,0.980757
206831,2025-12-21,[미리보는 이데일리 신문]차세대 비만약 핵심기술 K바이오가 장악,[이데일리 박지애 기자] 다음은 22일자 이데일리 신문 주요 기사다. △1면 -...,https://n.news.naver.com/mnews/article/018/000...,헬스케어,0.364675


In [15]:
df['날짜'] = pd.to_datetime(df['날짜'])

In [16]:
df['날짜'] = df['날짜'].dt.strftime('%Y-%m-%d')

In [17]:
df.to_pickle('sentiment.pickle')

In [18]:
df

,날짜,제목,본문,링크,산업,sentiment_score
9,2025-10-01,"캐너코드,""테슬라 3분기 인도 48만대 넘어""…목표주가도 상향",이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. ...,https://n.news.naver.com/mnews/article/015/000...,자동차,0.855592
16,2025-10-01,트럼프 큰소리에도 美자동차 업체는 공장 구조조정중,이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. ...,https://n.news.naver.com/mnews/article/015/000...,자동차,0.449221
19,2025-10-01,[이슈프리즘] 韓 제조업 미래는 누가 고민하나,“중국이 제조 혁신에 나섰다고 화웨이 같은 회사가 자동차 등 다른 분야에서 곧 나올...,https://n.news.naver.com/mnews/article/015/000...,자동차,0.743420
47,2025-10-01,"""수소도시 만들어 수소산업 육성하자""",국회수소경제포럼 대표의원인 정태호 더불어민주당 의원이 수소산업 경쟁력 확보를 위한 ...,https://n.news.naver.com/mnews/article/008/000...,자동차,0.854099
48,2025-10-01,"국토부, 서울 중화5구역 점검… '공공 정비사업' 속도낸다",공공성 강화 중심의 '9·7 공급대책'을 내놓은 정부가 공공정비사업 사업성을 확대하...,https://n.news.naver.com/mnews/article/008/000...,건설,0.847423
...,...,...,...,...,...,...
206635,2025-12-21,3200명 정신없는 건설현장… AI·통합관제로 안전 지킨다[현장르포],지난 19일 찾은 서울 서초구 디에이치 클래스트(반포주공1단지 1·2·4주구 재건축...,https://n.news.naver.com/mnews/article/014/000...,건설,0.963720
206751,2025-12-21,[AI헬스케어] 정부 주도 AI의료 기술 개발...닥터앤서3.0 참여 기업은,[이데일리 김승권 기자] 정부 주도 의료 인공지능(AI) 사업 ‘닥터앤서’가 3.0...,https://n.news.naver.com/mnews/article/018/000...,헬스케어,0.882646
206814,2025-12-21,살 뺄 때만 먹는 줄 알았는데…알고보니 '완전식품'인 고구마의 놀라운 효능 [건강UP],[서울경제] 몸·마음·생활… 무분별한 정보는 많고 건강해야 할 곳도 많습니다. ...,https://n.news.naver.com/mnews/article/011/000...,헬스케어,0.980757
206831,2025-12-21,[미리보는 이데일리 신문]차세대 비만약 핵심기술 K바이오가 장악,[이데일리 박지애 기자] 다음은 22일자 이데일리 신문 주요 기사다. △1면 -...,https://n.news.naver.com/mnews/article/018/000...,헬스케어,0.364675


In [19]:
df.to_pickle('sentiment.pickle')